In [155]:
# Imports
%matplotlib inline


from My_Functions import * 
import numpy as np
import pandas as pd
import seaborn as sns
import warnings

from collections import defaultdict

from sklearn import preprocessing
from sklearn.metrics import mean_squared_error, r2_score,roc_curve
from sklearn.model_selection import train_test_split, KFold,StratifiedKFold
from sklearn.model_selection import cross_val_score, cross_val_predict,validation_curve
from sklearn.ensemble import RandomForestRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.linear_model import SGDRegressor

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelBinarizer, RobustScaler,PolynomialFeatures
from sklearn.neighbors import KNeighborsRegressor
from scipy import stats
from scipy.stats import skew, boxcox_normmax
from scipy.special import boxcox1p

import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator , MultipleLocator
##from matplotlib import XAxis
from gplearn.genetic import SymbolicRegressor

from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, LabelBinarizer,MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import RFE, RFECV
from sklearn.linear_model import LogisticRegression,LinearRegression, OrthogonalMatchingPursuit
from sklearn.model_selection import train_test_split , TimeSeriesSplit, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score
from matplotlib.gridspec import GridSpec
import plotly.tools as tls
import plotly
plotly.tools.set_credentials_file(username='Furqan92', api_key='22DfVN5rFRg79OYygN5h')
import plotly.plotly as py
from sklearn.decomposition import PCA
from pandas import DataFrame 
from sklearn.exceptions import ConvergenceWarning
import warnings

warnings.simplefilter(action='ignore')
random_seed = 504
tscv = TimeSeriesSplit(n_splits=5)

In [156]:
# Load datasets

wd_h=pd.read_csv('./workingdays_data_prepared.csv')
#we_h=pd.read_csv('./weekends_holi_data_prepared.csv')
wd_h['dteday']=pd.to_datetime(wd_h['dteday'], format='%Y-%m-%d')
#we_h['dteday']=pd.to_datetime(we_h['dteday'], format='%Y-%m-%d')

## Subset 2011 -> 2012Q3 Data

#### Dividing the dataset in one year and a half and Q3 iun order to be able to train level 0 AND PREDICT

In [157]:
X_Train_2011_2012Q2 = wd_h[wd_h['dteday'] < datetime.datetime(2012, 7, 5, 0, 0)].drop(['cnt','casual','registered','dteday'],axis=1)## NONE OF THst_EM IN DATA
Y_cnt_train_2011_2012Q2 =wd_h['cnt'][wd_h['dteday'] < datetime.datetime(2012, 7, 5, 0, 0)]

X_Test_2012Q3 = wd_h[(wd_h['dteday'] >= datetime.datetime(2012, 7, 5, 0, 0)) & (wd_h['dteday'] <= datetime.datetime(2012, 9, 30, 0, 0))].drop(['cnt','casual','registered','dteday'],axis=1)## NONE OF THEM IN DATA
Y_cnt_test_2012Q3 =wd_h['cnt'][(wd_h['dteday'] >= datetime.datetime(2012, 7, 5, 0, 0)) & (wd_h['dteday'] <= datetime.datetime(2012, 9, 30, 0, 0))]

## Linear regression w/ GS for *Weekdays* for 2011 -> 2012Q3

Starting with the simplest model on Q3 , even though the score is high here this is the prediction for Q3 (FALL) , as we saw in the EDA FALL SPRING AND SUMMER behaved in the same way but winter was a bit different so we can say that the model learned from 5 similar season and we would not expect the same score for Q4 ( winter ) . This clearly an indicator that the feature created previously have helped our model to learn more about the target

In [159]:
##Linear Regression
lm_parameters = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True]}

lm = GridSearchCV(LinearRegression(),
                                 param_grid=lm_parameters,
                                 cv=tscv,return_train_score=True)

lm.fit(X_Train_2011_2012Q2, Y_cnt_train_2011_2012Q2)
lm.cv_results_
lm_predictions = lm.predict(X_Test_2012Q3)
lm.score(X_Test_2012Q3, Y_cnt_test_2012Q3)



0.9408507786648361

## Random Forest w/ GS for *Weekdays* for 2011 -> 2012Q3

After trying linear regressor we tried random forest with a large grid search in order to make sure that we are using the best possible parameters for the model 
Our expectations were to have a higher score then the linear model but we did not, This could be due to the fact that there are alot of linearly correlated target to the data such as the new weather variable created and the 
mean_per_hour which reflects a direct linear correlation of the past target in the same hour.
Random forest acts best when the data is clearly partitione in groups such as the Rush_Hour and Is_Daylight that clearly differentiate the different target values

In [160]:
RF_parameters = {'n_estimators': [10, 30 ,100],
                                             'bootstrap': [True],
                                             'max_depth': [80, 100 ],
                                             'max_features': ['sqrt',16 ,32],
                                             'min_samples_leaf': [2,  5 , 8],
                                             'min_samples_split': [ 10 , 8 , 15],
                                            'random_state':[random_seed],
                                            'criterion':['mse']}
rf = GridSearchCV(RandomForestRegressor(),
                                 param_grid= RF_parameters,
                                 cv=tscv)

rf.fit(X_Train_2011_2012Q2, Y_cnt_train_2011_2012Q2)
rf.cv_results_
rf_predictions = lm.predict(X_Test_2012Q3)
rf.score(X_Test_2012Q3, Y_cnt_test_2012Q3)


0.9289808963621973

## K Nearest Neighbors regressor w/ GS for *Weekends* for 2011 -> 2012Q3

In [ ]:
After trying the basic model we went into the more exotic ones , here the KNN claculate the difference between the differernt point and tries to classsif them 
in groups and then compare these groups to the target and trie to get teh cclosest estimated number.
The bases of trying this model and why it seemed like it would make sense to us is because in our EDA it looked like there are multiple waya to cluster the differnret hours 
either by weather , hr ,season or even day  

In [161]:
knn_parameters = {'n_neighbors':[5,10,15],'weights':['uniform'], 'algorithm':['auto'],'leaf_size':[10, 20,30],
                  'p':[1,2,3],'metric':['minkowski']}
knn = GridSearchCV(KNeighborsRegressor(),
                                 param_grid=knn_parameters,
                                 cv=tscv,return_train_score=True)
knn.fit(X_Train_2011_2012Q2, Y_cnt_train_2011_2012Q2)
# rf.cv_results_
knn_predictions = knn.predict(X_Test_2012Q3)
knn.score(X_Test_2012Q3, Y_cnt_test_2012Q3)

0.9211277733616634

## Support Vector regressor w/ GS for *Weekends* for 2011 -> 2012Q3

In [ ]:
This algorithm is known for it complexity and the amount of time it takes oto run but we wanted to give it a try , so instead of using a wide range for the gridsearc we just used one of the basic kernels 

In [162]:

svr = GridSearchCV(SVR(kernel='rbf', gamma=0.1), cv=tscv,
                   param_grid={"C": [1e0, 1e1, 1e2, 1e3],
                               "gamma": np.logspace(-2, 2, 5)})

svr.fit(X_Train_2011_2012Q2, Y_cnt_train_2011_2012Q2)
# rf.cv_results_
svr_predictions = svr.predict(X_Test_2012Q3)
svr.score(X_Test_2012Q3, Y_cnt_test_2012Q3)


0.7455187270287198

## XGBoost regressor w/ GS for *Weekends* for 2011 -> 2012Q3

In [163]:


param_grid = {'learning_rate': [0.01, 0.1], 
          'max_depth': [4,8,12],
          'min_child_weight': [3,5,10,20,35,50],
          'subsample': [0.5, 0.75],
          'colsample_bytree': [0.5, 0.75],
          'n_estimators': [100, 300], 'random_state':[random_seed]
}

model = xgb.XGBRegressor()

xg = GridSearchCV(model,
                    param_grid = param_grid,
                    cv = tscv,
                    n_jobs = -1,
                    scoring = 'explained_variance',
                    verbose=True)
xg.fit(X_Train_2011_2012Q2, Y_cnt_train_2011_2012Q2)
xg_predictions = xg.predict(X_Test_2012Q3)
xg.score(X_Test_2012Q3, Y_cnt_test_2012Q3)

Fitting 5 folds for each of 288 candidates, totalling 1440 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   28.5s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  7.7min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed: 14.5min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed: 24.6min
[Parallel(n_jobs=-1)]: Done 1440 out of 1440 | elapsed: 30.6min finished


0.9430402871403295

## Taking predictions from LR and Random Forest and xgboost to test on 2012Q4

In [164]:
# LR and Random Forest predictions
combinedPredictions = pd.DataFrame({'lm_predictions':lm_predictions,'rf_predictions':rf_predictions,'xg_predictions':xg_predictions })

# Concat original data
X_Test_2012Q3.reset_index(drop=True,inplace=True)
combinedPredictions = pd.concat([combinedPredictions,X_Test_2012Q3], axis=1)

# Target data
combinedPredictionsTarget = pd.DataFrame({'target':Y_cnt_test_2012Q3})

#Reset indices
combinedPredictions.reset_index(drop=True,inplace=True)
combinedPredictionsTarget.reset_index(drop=True,inplace=True)

## Subset Q4 data and concat it with its level-0 predctions

In [165]:
#Get Q4 data
X_Test_Q4 = wd_h[(wd_h['dteday'] >= datetime.datetime(2012, 10, 1, 0, 0)) & (wd_h['dteday'] <= datetime.datetime(2012, 12, 31, 0, 0))].drop(['cnt','casual','registered','dteday'],axis=1)## NONE OF THEM IN DATA
Y_cnt_test_Q4 =wd_h['cnt'][(wd_h['dteday'] >= datetime.datetime(2012, 10, 1, 0, 0)) & (wd_h['dteday'] <= datetime.datetime(2012, 12, 31, 0, 0))]
oringal_cnt = Y_cnt_test_Q4

#get level-0 predictions for Q4 data
Q4_lm_predictions = pd.DataFrame(lm.predict(X_Test_Q4), columns=["lm_predictions"])
Q4_rf_predictions = pd.DataFrame(rf.predict(X_Test_Q4), columns=["rf_predictions"])
Q4_xg_predictions = pd.DataFrame(xg.predict(X_Test_Q4), columns=["xg_predictions"])

X_Test_Q4.reset_index(drop=True,inplace=True)
Q4_lm_predictions.reset_index(drop=True,inplace=True)
Q4_rf_predictions.reset_index(drop=True,inplace=True)
Q4_xg_predictions.reset_index(drop=True,inplace=True)
Y_cnt_test_Q4.reset_index(drop=True,inplace=True)

#concat Q4 with level-0 predictions 
X_Test_Q4_with_predictions = pd.concat([Q4_lm_predictions, Q4_rf_predictions,Q4_xg_predictions, X_Test_Q4 ], axis=1)
Y_cnt_test_Q4 = pd.DataFrame({"target": Y_cnt_test_Q4})

## Q4 scores

In [166]:
lm.score(X_Test_Q4, Y_cnt_test_Q4)

0.8839966859540709

In [167]:
rf.score(X_Test_Q4, Y_cnt_test_Q4)

0.8715926918607073

In [168]:
xg.score(X_Test_Q4, Y_cnt_test_Q4)

0.900164954326128

In [169]:
knn.score(X_Test_Q4, Y_cnt_test_Q4)

0.8340477314724115

In [170]:
svr.score(X_Test_Q4, Y_cnt_test_Q4)

0.7077025873956615

## Train and test Q4

### XGBoost regressor w/ GS for Level 1 *Weekends* for 2011 -> 2012Q3

In [171]:
param_grid = {'learning_rate': [0.01, 0.1], 
          'max_depth': [4,8,12],
          'min_child_weight': [3,5,10,20,35,50],
          'subsample': [0.5, 0.75],
          'colsample_bytree': [0.5, 0.75],
          'n_estimators': [100, 300], 'random_state':[random_seed]
}

model = xgb.XGBRegressor()

xg1 = GridSearchCV(model,
                    param_grid = param_grid,
                    cv = tscv,
                    n_jobs = -1,
                    scoring = 'explained_variance',
                    verbose=True)

xg1.fit(combinedPredictions, combinedPredictionsTarget)
xg1_predictions = xg1.predict(X_Test_Q4_with_predictions)
xg1.score(X_Test_Q4_with_predictions, Y_cnt_test_Q4)

Fitting 5 folds for each of 288 candidates, totalling 1440 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  51 tasks      | elapsed:    6.7s
[Parallel(n_jobs=-1)]: Done 204 tasks      | elapsed:   31.1s
[Parallel(n_jobs=-1)]: Done 454 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 804 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 1254 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 1440 out of 1440 | elapsed:  4.8min finished


0.9022288970285075

In [172]:
xg1_predictions  = pd.concat( (pd.DataFrame(xg1.predict(X_Test_Q4_with_predictions)), Y_cnt_test_Q4), axis =1 )
xg1_predictions.to_csv("pred_working.csv", index=False)